# Running the Luminosity-Temperature-Radius pipeline

We will use the XGA LTR pipeline to try to determine overdensity radii, and the global temperature and luminosity measurements within the other structures we have noted in our observations. This process is [fully described here](https://xga.readthedocs.io/en/latest/notebooks/pipeline_tutorials/LT_pipeline.html).

## Import statements

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from astropy.units import Quantity
import os
from shutil import rmtree

import xga
# This just sets the number of cores this analysis is allowed to use
xga.NUM_CORES = 40
# This is a bodge that will only work because xga_output in notebooks has already been defined, XGA
#  will be made to handle this more gracefully at some point
temp_dir = xga.OUTPUT
actual_dir = temp_dir.split('notebooks/lum_temp_rad_pipeline')[0]+'xga_output/'
xga.OUTPUT = actual_dir
xga.utils.OUTPUT = actual_dir
# As currently XGA will setup an xga_output directory in our current directory, I remove it to keep it all clean
if os.path.exists('xga_output'):
    rmtree('xga_output')
from xga.tools import luminosity_temperature_pipeline
from xga.relations.clusters.RT import arnaud_r2500

%matplotlib inline

## Reading the sample file

Reading in the positions, names, and redshifts of the other structures that we have noted - the pipeline expects position columns to be called 'ra' and 'dec', so those are renamed:

In [2]:
samp = pd.read_csv('../../sample_files/poss_assoc_struc.csv')
samp = samp.rename(columns={'est_ra': 'ra', 'est_dec': 'dec'})
samp

,name,ra,dec,redshift
0,12833B,129.445090,44.378633,0.147
1,PHL1811B,328.655285,-9.259118,0.078
2,25124B,243.505797,26.687077,0.187


## Running the pipeline (twice)

The LTR will be run twice, targeting two different overdensity radii. The pipeline runs will save the results to disk (as we are passing paths for the results and radii history files), as well as returning the results and the XGA ClusterSample object generated by the procedure.

### $R_{500}$

The first overdensity radius we are targeting is $R_{500}$, which is the default for this pipeline. The Arnaud et al. (2005) $R_{500}$-$T_{\rm{X}}$ is used for the process, and the starting radius is set to 200 kpc. Metallicity, nH, and redshift are all frozen during the spectral fitting process.

In [3]:
res_file = '../../outputs/results/other_struct_ltr_r500_pipeline_results.csv'
rhist_file = '../../outputs/results/other_struct_radii_r500_history.csv'

srcs, results, rad_hist = luminosity_temperature_pipeline(samp, Quantity(200, 'kpc'), timeout=Quantity(4, 'hr'), 
                                                          save_samp_results_path=res_file,
                                                          save_rad_history_path=rhist_file, 
                                                          core_excised=True)

Running XSPEC Fits: 100%|██████████| 3/3 [00:39<00:00, 13.12s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/products/relation.py:704: UserWarning: Some of the x values you have passed are outside the validity range of this relation (1.0-12.0keV).
  warn("Some of the x values you have passed are outside the validity range of this relation "
Running XSPEC Fits: 100%|██████████| 2/2 [01:18<00:00, 39.27s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/tools/clusters/LT.py:297: UserWarning: The radius measurement process reached the maximum number of iterations; as such one or more clusters may have unconverged radii.
  warn("The radius measurement process reached the maximum number of iterations; as such one or more clusters "
Running XSPEC Fits: 100%|██████████| 3/3 [00:49<00:00, 16.57s/it]


### $R_{2500}$

The other overdensity radius we are targeting is $R_{2500}$. The Arnaud et al. (2005) $R_{2500}$-$T_{\rm{X}}$ is used for the process. As the ClusterSample in this run is completely separate from the $R_{500}$ run, it is possible that slightly different regions have been selected as contaminating sources, this would be due to XGA ClusterSamples allowing point sources in the core region to not be masked out, in case they are misidentified cool-cores.

In [4]:
res_file = '../../outputs/results/other_struct_ltr_r2500_pipeline_results.csv'
rhist_file = '../../outputs/results/other_struct_radii_r2500_history.csv'

srcs_r2500, results_r2500, \
    rad_hist_r2500 = luminosity_temperature_pipeline(samp, Quantity(200, 'kpc'), timeout=Quantity(4, 'hr'), 
                                                     rad_temp_rel=arnaud_r2500, save_samp_results_path=res_file,
                                                     save_rad_history_path=rhist_file)

Setting up Galaxy Clusters: 100%|██████████| 3/3 [00:02<00:00,  1.19it/s]
/mnt/ufs18/home-218/turne540/code/XGA/xga/samples/extended.py:237: UserWarning: Non-fatal warnings occurred during the declaration of some sources, to access them please use the suppressed_warnings property of this sample.
  self._check_source_warnings()
Running XSPEC Fits: 100%|██████████| 3/3 [00:39<00:00, 13.04s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/products/relation.py:704: UserWarning: Some of the x values you have passed are outside the validity range of this relation (1.0-12.0keV).
  warn("Some of the x values you have passed are outside the validity range of this relation "
Running XSPEC Fits: 100%|██████████| 3/3 [00:40<00:00, 13.65s/it]
/mnt/ufs18/home-218/turne540/code/XGA/xga/products/relation.py:704: UserWarning: Some of the x values you have passed are outside the validity range of this relation (1.0-12.0keV).
  warn("Some of the x values you have passed are outside the validity range of thi

## Pipeline results

We take a look at the tables of results output by the pipeline runs:

### $R_{500}$

In [5]:
results

,name,ra,dec,redshift,r500,Tx500,Tx500-,Tx500+,Lx500_0.5-2.0,Lx500_0.5-2.0-,Lx500_0.5-2.0+,Lx500_0.01-100.0,Lx500_0.01-100.0-,Lx500_0.01-100.0+,Tx500ce,Tx500ce-,Tx500ce+,Lx500ce_0.5-2.0,Lx500ce_0.5-2.0-,Lx500ce_0.5-2.0+,Lx500ce_0.01-100.0,Lx500ce_0.01-100.0-,Lx500ce_0.01-100.0+
0,12833B,129.445090,44.378633,0.147,446.154488,1.198340,0.102703,0.146824,2.683993e+42,2.817973e+41,3.821836e+41,5.041254e+42,6.860842e+41,7.319454e+41,0.942106,0.115195,0.144415,1.422721e+42,3.167232e+41,2.465148e+41,2.398064e+42,5.014045e+41,4.721315e+41
1,PHL1811B,328.655285,-9.259118,0.078,218.280606,0.861658,0.040658,0.042252,6.897264e+41,9.353338e+40,1.070696e+41,1.128981e+42,1.269325e+41,1.483954e+41,0.844760,0.060697,0.061263,4.507643e+41,6.214318e+40,7.064221e+40,7.332803e+41,1.348299e+41,1.266887e+41
2,25124B,243.505797,26.687077,0.187,703.981490,0.975398,0.154552,0.201960,2.141008e+42,5.212579e+41,6.433501e+41,3.650635e+42,9.961569e+41,1.106682e+42,0.958811,0.238586,0.258457,1.235474e+42,7.251886e+41,4.409429e+41,2.091555e+42,1.240169e+42,8.637601e+41


In [6]:
rad_hist

,0,1,2,3,4,5,6,7,8,9,10,converged
12833B,200.0,463.237415,428.234087,451.897062,446.154488,446.154488,446.154488,446.154488,446.154488,446.154488,446.154488,True
PHL1811B,200.0,410.485679,218.222062,390.131706,218.129811,391.439959,217.725967,391.976723,218.246925,389.777654,218.280606,False
25124B,200.0,604.288516,827.651244,492.573040,657.671188,755.332054,410.688173,785.529488,510.197971,654.525753,703.981490,True


### $R_{2500}$

In [7]:
results_r2500

,name,ra,dec,redshift,r2500,Tx2500,Tx2500-,Tx2500+,Lx2500_0.5-2.0,Lx2500_0.5-2.0-,Lx2500_0.5-2.0+,Lx2500_0.01-100.0,Lx2500_0.01-100.0-,Lx2500_0.01-100.0+
0,12833B,129.445090,44.378633,0.147,207.168987,1.173990,0.071080,0.079994,2.008451e+42,1.884364e+41,1.667938e+41,3.737629e+42,3.050789e+41,5.117773e+41
1,PHL1811B,328.655285,-9.259118,0.078,187.472641,0.959488,0.039578,0.043469,5.201245e+41,7.827132e+40,9.164976e+40,8.867894e+41,1.168027e+41,1.359024e+41
2,25124B,243.505797,26.687077,0.187,327.624927,4.333690,1.633110,4.424408,2.188118e+42,3.601451e+41,3.546524e+41,6.675148e+42,1.705071e+42,1.510451e+42


In [8]:
rad_hist_r2500

,0,1,2,3,4,5,6,7,8,9,10,converged
12833B,200.0,208.925578,206.409290,203.478940,207.168987,207.168987,207.168987,207.168987,207.168987,207.168987,207.168987,True
PHL1811B,200.0,185.639379,186.937657,187.782224,187.472641,187.472641,187.472641,187.472641,187.472641,187.472641,187.472641,True
25124B,200.0,271.174188,288.381330,312.537793,381.125739,142.752653,327.669178,413.586350,350.763544,142.764972,327.624927,False
